<b>Plant Location</b>

<img src="plant location.jpg" width=55% align="left">

### Plant Location:

**Summary:** The goal is to open new plants to satisfy demands from three customers at the minimum cost.

***Let us denote:***

- $P$ as the set of all plants (from Plant 1 to Plant 5 in this case).
- $C$ as the set of all customers (A, B, C in this case).
- $c_{i}$ as the fixed cost of opening plant $i$, for $i \in P$.
- $p_{ij}$ as the production/transportation cost to supply customer $j$ from plant $i$, for $i \in P$, $j \in C$.
- $D_{j}$ as the demand of customer $j$, for $j \in C$.
- $K_{i}$ as the capacity of plant $i$, for $i \in P$.
- $x_{i}$ as the binary decision variable which equals 1 if plant $i$ is opened, 0 otherwise, for $i \in P$.
- $y_{ij}$ as the amount shipped from plant $i$ to customer $j$, for $i \in P$, $j \in C$.

**Objective Function:**

Minimize the total fixed and supply costs:

$$\min \sum_{i \in P} c_{i} \cdot x_{i} + \sum_{i \in P} \sum_{j \in C} p_{ij} \cdot y_{ij}$$

**Constraints:**

1. `Meet Demand:` The total amount supplied to each customer should meet their demand:

$$\sum_{i \in P} y_{ij} = D_{j} \quad for \: all \: j \in C $$

2. `Capacity Constraint:` The total amount supplied from each plant should not exceed its capacity, also linking the usage of $y_{ij}$ and $x_{i}$:

$$\sum_{j \in C} y_{ij} \le K_{i} \cdot x_{i} \quad for \: all \: i \in P $$

3. `Non-negativity and Binary Constraints:` The amounts supplied are non-negative and the decision to open a plant is binary:

$$y_{ij} \ge 0 \quad for \: all \: i \in P, j \in C$$

$$x_{i} \in \{0,1\} \quad for \: all \: i \in P$$

In this problem, we aim to minimize the total cost of opening plants and supplying customers while meeting the demands of each customer and not exceeding the capacity of each plant. The solution indicates which plants should be opened and how much each plant should supply to each customer to achieve this objective.


In [1]:
# data
FixedCost = {1 : 1325000, 2 : 1100000, 3 : 1500000, 4 : 1200000, 5 : 1400000}
PlantCapacity = {1 : 40000, 2 : 30000, 3 : 50000, 4 : 20000, 5 : 40000}
SupplyCost = {
    (1,'A') : 35, 	(1,'B') : 30, 	(1,'C') : 45, 
    (2,'A') : 45, 	(2,'B') : 40, 	(2,'C') : 50, 
    (3,'A') : 70, 	(3,'B') : 65, 	(3,'C') : 50, 
    (4,'A') : 20, 	(4,'B') : 45, 	(4,'C') : 25, 
    (5,'A') : 65, 	(5,'B') : 45, 	(5,'C') : 45
}
Demand = {'A' : 40000, 'B' : 25000, 'C' : 35000}

In [2]:
Plants = FixedCost.keys()
Customers = Demand.keys()

In [3]:
from docplex.mp.model import Model
mdl = Model()

In [4]:
# variables
select = mdl.binary_var_dict(Plants, name='select')
supply = mdl.continuous_var_matrix(Plants, Customers, lb=0, name='supply')

In [5]:
# objective
TotalFixedCost = mdl.sum(FixedCost[i]*select[i] for i in Plants)
TotalSupplyCost = mdl.sum(SupplyCost[i,j]*supply[i,j] for (i,j) in supply)
mdl.minimize(TotalFixedCost + TotalSupplyCost)

# add KPIs
mdl.add_kpi(TotalFixedCost, 'Total fixed cost')
mdl.add_kpi(TotalSupplyCost, 'Total supply cost')

DecisionKPI(name=Total supply cost,expr=35supply_1_A+30supply_1_B+45supply_1_C+45supply_2_A+40supply_2_B..)

In [6]:
# constraints: meet demand
for j in Customers:
    mdl.add_constraint(mdl.sum(supply[i,j] for i in Plants) == Demand[j])

In [7]:
# constraints: capacity and linking constraints
for i in Plants:
    mdl.add_constraint(mdl.sum(supply[i,j] for j in Customers) <= PlantCapacity[i]*select[i])

In [8]:
# solve
mdl.solve()
mdl.get_solve_details()

docplex.mp.SolveDetails(time=0.062,status='integer optimal solution')

In [9]:
mdl.print_solution()

objective: 7425000.000
  select_1=1
  select_4=1
  select_5=1
  supply_1_A=20000.000
  supply_1_B=20000.000
  supply_4_A=20000.000
  supply_5_B=5000.000
  supply_5_C=35000.000


In [10]:
mdl.report()

* model docplex_model1 solved with objective = 7425000.000
*  KPI: Total fixed cost  = 3925000.000
*  KPI: Total supply cost = 3500000.000
